In [ ]:
"""analisis de malware y benigno con dataset y multilayer
preprocesado por chinos con binario
paper 244802 en df 293333
https://datascience.stackexchange.com/questions/36049/how-to-adjust-the-hyperparameters-
of-mlp-classifier-to-get-more-perfect-performa
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix 
from sklearn import datasets, metrics
from sklearn.metrics import classification_report 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import os
import glob

"""crea entorno grÃ¡fico"""
plt.rcParams['figure.figsize'] = (40, 50)
plt.style.use('ggplot')

# Primero especificamos un patrón del archivo y lo pasamos como parámetro en la función glob
os.chdir("..\\Datasets")
os.getcwd()
csv_files = glob.glob('*.csv')
# Mostrar el archivo csv_files, el cual es una lista de nombres
print(csv_files)

list_data = []
  
# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será la lista dataframes

for filename in csv_files:
    data = pd.read_csv(filename)
    list_data.append(data)

#Para chequear que todo está bien, mostramos la list_data por consola
#list_data
 
df = pd.concat(list_data,ignore_index=True)

tipos=df.dtypes  #tipos de datos de campos

"""información datos"""
print(df.head(5))
print('Cantidad de Filas y columnas:',df.shape)
print('Nombre columnas:',df.columns)

"""Columnas, nulos y tipo de datos"""
print(df.info())

"""descripción estadística de los datos numéricos"""
print(df.describe())

"""correlación entre los datos"""
corr = df.set_index('Label').corr()
#sm.graphics.plot_corr(corr, xnames=list(corr.columns))
#plt.show()

df2=df[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Min',
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']]
df3=df[['Label']]

"""convertir columna tipo dato str a int"""
df2['Label']=df3[['Label']]

"""desordena el dataset"""

df2=df2.sort_values('Fwd_Pkt_Len_Min')
df2 = df2[df2['Label'].isin([1,2,3,4])]

""" convertir valores número la clase"""
df2['Label'] = df2['Label'].replace(1,"Benigno")
df2['Label'] = df2['Label'].replace(2,"Malware")
df2['Label'] = df2['Label'].replace(3,"Malware")
df2['Label'] = df2['Label'].replace(4,"Malware")
#print(df2.head(10))
#print(df2.groupby('Label').size())


dff=df2[:]
dff1 = dff[dff.Label == 'Malware']
dff = dff.drop(dff[dff['Label'] == 'Malware'].index)
a = dff1.iloc[0:134800,]
df2 = pd.concat([dff,a])



print(df2.groupby('Label').size())




"""CREAMOS EL MODELO Y LO PROBAMOS CON LOS MISMOS DATOS SIN CROSS"""


# Split dataset in training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(df2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],df2['Label'], test_size=0.2, random_state=6) 


    
from sklearn.neural_network import MLPClassifier



clf =MLPClassifier(max_iter=60)


clf.fit(X_train, Y_train)


accuracy = clf.score(X_train, Y_train) *100
print("Precision entrenamiento: "+ str(accuracy))

predictions = clf.predict(X_test) 
  
# model accuracy for X_test   
accuracy = clf.score(X_test, Y_test) *100
print("Precision validado: "+ str(accuracy))
# creating a confusion matrix 
cm = confusion_matrix(Y_test, predictions) 
print(cm)
print("Métricas")
print(classification_report(Y_test, predictions))

""" convierto valor nombre a 0=benigno 1=malware para AUC-ROC"""

label_encoder = LabelEncoder()
df2.iloc[:,15] = label_encoder.fit_transform(df2.iloc[:,15]).astype('int64')
X_train, X_test, Y_train, Y_test = train_test_split(df2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],df2['Label'], test_size=0.2, random_state=6) 


def plot_roc(model, X_test, Y_test):
    # calculate the fpr and tpr for all thresholds of the classification
    probabilities = model.predict_proba(np.array(X_test))
    predictions = probabilities[:, 1]
    fpr, tpr, threshold = metrics.roc_curve(Y_test, predictions)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

plot_roc(clf, X_test, Y_test)


"""============================================="""
""" CROSS VALIDATION"""

X_trainCV, X_testCV, Y_trainCV, Y_testCV = train_test_split(df2[['Flow_Duration', 'Tot_Fwd_Pkts', 'Tot_Bwd_Pkts', 'TotLen_Fwd_Pkts',
       'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Min', 
       'Fwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Max', 'Fwd_Pkt_Len_Mean',
       'Bwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Flow_Pkts/s',
       'Flow_Byts/s']],df2['Label'], test_size=0.2, random_state=6) 


clfCV = MLPClassifier(max_iter=60)
clfCV.fit(X_trainCV, Y_trainCV)
kf = KFold(n_splits=10)
score = clfCV.score(X_trainCV,Y_trainCV)
print("Metrica del modelo", score * 100)
scores = cross_val_score(clfCV, X_trainCV, Y_trainCV, cv=kf, scoring="accuracy")
print("Metricas cross_validation", scores * 100)
#print("Media de cross_validation", scores.mean() * 100)
preds = clfCV.predict(X_testCV)
score_pred = metrics.accuracy_score(Y_testCV, preds)
print("Metrica en Test", score_pred * 100)
conf_mat = confusion_matrix(Y_testCV, preds)
print(conf_mat)
print(classification_report(Y_testCV, preds))

def plot_roc(model, X_test, Y_test):
    # calculate the fpr and tpr for all thresholds of the classification
    probabilities = model.predict_proba(np.array(X_test))
    predictions = probabilities[:, 1]
    fpr, tpr, threshold = metrics.roc_curve(Y_test, predictions)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

plot_roc(clfCV, X_testCV, Y_testCV)




"""
sin
Label
Benigno    137907
Malware    155426
dtype: int64
Precision entrenamiento: 78.16428455762659
Precision validado: 78.23307822114647
[[18597  8865]
 [ 3905 27300]]
Métricas
              precision    recall  f1-score   support
     Benigno       0.83      0.68      0.74     27462
     Malware       0.75      0.87      0.81     31205
    accuracy                           0.78     58667
   macro avg       0.79      0.78      0.78     58667
weighted avg       0.79      0.78      0.78     58667
Label
Benigno    137907
Malware    155426
dtype: int64
Precision entrenamiento: 94.57612095488908
Precision validado: 94.51309935739002
[[25362  2194]
 [ 1025 30086]]
Métricas
              precision    recall  f1-score   support
     Benigno       0.96      0.92      0.94     27556
     Malware       0.93      0.97      0.95     31111
    accuracy                           0.95     58667
   macro avg       0.95      0.94      0.94     58667
weighted avg       0.95      0.95      0.95     58667
"""